# Install Necessary Dependencies

In [1]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install -q transformers accelerate peft datasets
!pip install -U bitsandbytes trl

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-r426tka8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-r426tka8

  Resolved https://github.com/huggingface/transformers.git to commit ba29a439adbe6f371710d0514659127264ae24b3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 212.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.9/780.9 kB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 161.8 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.49.0.dev0-py3-none-any.whl size=10719728 sha256=24c34aa7bcbae784ab95e1f8b801242d8302f153f613c84aeb763d7f393ff7e0
  Stored in directory: /tmp/pip-ephem-wheel

In [2]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 136.6 MB/s eta 0:00:00


# Import

In [3]:
import os
import torch
import warnings
import transformers
from trl import SFTTrainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub import HfApi, HfFolder, Repository, notebook_login

### Login to Hugging Face using Access Token

In [4]:
notebook_login()

# Load your Fine-Tuned Model from Hugging Face(models)

In [6]:
model_name = "sagarvk24/Meta-Llama-3.1-8B-Finance-FineTune-Sagar"

## Perform Quantization for efficient resource utilisation

In [7]:
# Define quantization configuration
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16,  # Use FP16 for computation
    bnb_4bit_use_double_quant=True,  # Further reduces memory usage
)

In [8]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto"  # Automatically assigns model to available GPU
)

model-00001-of-00004.safetensors:  34%|###3      | 1.68G/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

# Let's Check Inferences

In [ ]:
input_text = "What are the key financial trends in 2025?"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to("cuda")
output = model.generate(**inputs, max_new_tokens=50, temperature=0.7, top_p=0.9)
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True).strip()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [13]:
print(decoded_output)

What are the key financial trends in 2025? 2025 is expected to be a year of economic growth and recovery for the US and global economies. The Federal Reserve will likely continue to raise interest rates to manage inflation and maintain economic stability. The European Union is expected to face significant economic challenges due


In [14]:
# Test inference
input_text = "Tell me about SIP in easy manner."
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to("cuda")
output = model.generate(**inputs, max_new_tokens=50, temperature=0.7, top_p=0.9)
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True).strip()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [15]:
print(decoded_output)

Tell me about SIP in easy manner. SIP is a way of buying a small amount of shares on a regular basis. This is a long-term investment strategy. SIP is a way of buying a small amount of shares on a regular basis. This is a long-term investment strategy. SIP stands


In [16]:
# Test inference
input_text = "What are mutual funds? Are they safe?"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to("cuda")
output = model.generate(**inputs, max_new_tokens=50, temperature=0.7, top_p=0.9)
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True).strip()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [17]:
print(decoded_output)

What are mutual funds? Are they safe? Can I make money with them?
Mutual funds are a type of investment that pools money from many investors to invest in a variety of assets such as stocks, bonds, or other securities. They are managed by professional fund managers who aim to generate returns
